## Ce notebook permet de réaliser la détection d'une arme à feu sur une image ( lié à la page 1_Detection_Image.py)

#### Nous utiliserons un algorithme de Deep Learning, appelé CNN ( Convolutionnal Central Neural Networks) pour réaliser la classification des images multi-labels

In [107]:
# imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam


## Charger et préparer les données

https://www.kaggle.com/datasets/syedwajeeh19/weapon-detection-dataset-for-yolov3/data

In [95]:
# Charger les CSV contenant les labels pour chaque ensemble de données
train_labels_path = "C:/Users/33749/Desktop/IPSSI/MachineLearning_DeepLearning/Projet_Detection_Arme/Weapons.v1i.multiclass/train/_classes.csv"
test_labels_path = "C:/Users/33749/Desktop/IPSSI/MachineLearning_DeepLearning/Projet_Detection_Arme/Weapons.v1i.multiclass/test/_classes.csv"
validation_labels_path = "C:/Users/33749/Desktop/IPSSI/MachineLearning_DeepLearning/Projet_Detection_Arme/Weapons.v1i.multiclass/valid/_classes.csv"



In [99]:
# Charger les données pour l'ensemble d'entraînement, de validation et de test à partir des fichiers CSV
train_labels = pd.read_csv(train_labels_path, skipinitialspace=True)
test_labels = pd.read_csv(test_labels_path, skipinitialspace=True)
valid_labels = pd.read_csv(validation_labels_path, skipinitialspace=True)

In [100]:
# Chemins vers les répertoires contenant les images pour chaque ensemble de données
train_dir = "C:/Users/33749/Desktop/IPSSI/MachineLearning_DeepLearning/Projet_Detection_Arme/Weapons.v1i.multiclass/train/"
test_dir = "C:/Users/33749/Desktop/IPSSI/MachineLearning_DeepLearning/Projet_Detection_Arme/Weapons.v1i.multiclass/test/"
valid_dir = "C:/Users/33749/Desktop/IPSSI/MachineLearning_DeepLearning/Projet_Detection_Arme/Weapons.v1i.multiclass/valid/"


## Génération d'image

In [109]:

# # Paramètres des images
# image_size = (150, 150)
# batch_size = 32


# # Mise en place des générateurs d'images
# train_datagen = ImageDataGenerator(rescale=1./255)
# valid_datagen = ImageDataGenerator(rescale=1./255)
# test_datagen = ImageDataGenerator(rescale=1./255)

# train_generator = train_datagen.flow_from_dataframe(
#     dataframe=train_labels,
#     directory=train_dir,
#     x_col="filename",
#     y_col=["Gun","Knife","Rifle"],  # Utilisez des majuscules pour correspondre aux noms de colonnes dans le CSV
#     target_size=image_size,
#     batch_size=batch_size,
#     class_mode="raw"
# )

# valid_generator = valid_datagen.flow_from_dataframe(
#     dataframe=valid_labels,
#     directory=valid_dir,
#     x_col="filename",
#     y_col=["Gun","Knife","Rifle"],  # Utilisez des majuscules pour correspondre aux noms de colonnes dans le CSV
#     target_size=image_size,
#     batch_size=batch_size,
#     class_mode="raw"
# )

# test_generator = test_datagen.flow_from_dataframe(
#     dataframe=test_labels,
#     directory=test_dir,
#     x_col="filename",
#     y_col=["Gun","Knife","Rifle"],  # Utilisez des majuscules pour correspondre aux noms de colonnes dans le CSV
#     target_size=image_size,
#     batch_size=batch_size,
#     class_mode="raw"
# )


Found 35 validated image filenames.
Found 10 validated image filenames.
Found 5 validated image filenames.


## Création du modèle

In [111]:
# Paramètres des images
image_size = (150, 150)
batch_size = 32

# Mise en place des générateurs d'images
print("Chargement des données...")
print("Répertoire de formation:", train_dir)
print("Répertoire de validation:", valid_dir)
print("Répertoire de test:", test_dir)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_labels,
    directory=train_dir,
    x_col="filename",
    y_col=["Gun","Knife","Rifle"],
    target_size=image_size,
    batch_size=batch_size,
    class_mode="raw"
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid_labels,
    directory=valid_dir,
    x_col="filename",
    y_col=["Gun","Knife","Rifle"],
    target_size=image_size,
    batch_size=batch_size,
    class_mode="raw"
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_labels,
    directory=test_dir,
    x_col="filename",
    y_col=["Gun","Knife","Rifle"],
    target_size=image_size,
    batch_size=batch_size,
    class_mode="raw"
)

# Charger l'architecture InceptionV3 pré-entraînée sans les couches fully connected
print("Chargement du modèle InceptionV3...")
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Congeler les couches du modèle pré-entraîné pour empêcher leur mise à jour lors de l'entraînement
for layer in base_model.layers:
    layer.trainable = False

# Ajouter des couches fully connected personnalisées au-dessus de l'architecture pré-entraînée
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)  # 3 classes : Gun, Knife, Rifle

# Définir le modèle final
model = Model(inputs=base_model.input, outputs=predictions)

# Compiler le modèle
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Entraîner le modèle en utilisant les générateurs d'images
print("Entraînement du modèle...")
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=valid_generator,
    validation_steps=len(valid_generator)
)

# Évaluer le modèle sur l'ensemble de test
print("Évaluation du modèle sur l'ensemble de test...")
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print('\nTest accuracy:', test_acc)


Chargement des données...
Répertoire de formation: C:/Users/33749/Desktop/IPSSI/MachineLearning_DeepLearning/Projet_Detection_Arme/Weapons.v1i.multiclass/train/
Répertoire de validation: C:/Users/33749/Desktop/IPSSI/MachineLearning_DeepLearning/Projet_Detection_Arme/Weapons.v1i.multiclass/valid/
Répertoire de test: C:/Users/33749/Desktop/IPSSI/MachineLearning_DeepLearning/Projet_Detection_Arme/Weapons.v1i.multiclass/test/
Found 35 validated image filenames.
Found 10 validated image filenames.
Found 5 validated image filenames.
Chargement du modèle InceptionV3...
Entraînement du modèle...
Epoch 1/10
2/2 [==============================] - 5s 2s/step - loss: 10.8960 - accuracy: 0.6571 - val_loss: 6.9632 - val_accuracy: 0.8000
Epoch 2/10
2/2 [==============================] - 0s 382ms/step - loss: 4.7246 - accuracy: 0.7429 - val_loss: 2.1261 - val_accuracy: 0.7000
Epoch 3/10
2/2 [==============================] - 0s 346ms/step - loss: 4.2269 - accuracy: 0.5143 - val_loss: 3.0326 - val_accu

## Sauvegarde du modèle 

In [ ]:

model_save_path = r'C:\Users\33749\Desktop\IPSSI\MachineLearning_DeepLearning\Projet_Detection_Arme\detection_arme\Notebook\inceptionv3_model.h5'

model.save(model_save_path)e(model_save_path)

print("Le modèle a été sauvegardé avec succès à l'emplacement :", model_save_path)